In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 18085829
paper_name = 'alvaro_rothstein_2007' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [14]:
original_data = pd.read_excel('raw_data/pgen.0030228.st001.xlsx', sheet_name='SIA-Rad52 focus screen', skiprows=7)

In [15]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4818 x 7


In [16]:
original_data.head()

,Cells counted,# of Cells with Foci,% Cells with foci,ORF,Gene Name,Alias,Description
0,240,24,10,YMR056C,AAC1,NaN,Mitochondrial inner membrane ADP/ATP transloca...
1,201,3,1.49254,YBR085W,AAC3,ANC3,Mitochondrial inner membrane ADP/ATP transloca...
2,304,22,7.23684,YCR107W,AAD3,NaN,Putative aryl-alcohol dehydrogenase with simil...
3,190,26,13.6842,YDL243C,AAD4,NaN,Putative aryl-alcohol dehydrogenase with simil...
4,270,24,8.88889,YFL056C,AAD6,NaN,Putative aryl-alcohol dehydrogenase with simil...


In [17]:
original_data['orf'] = original_data['ORF'].astype(str)

In [18]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [21]:
original_data.loc[original_data['Alias']=='YLR006C','orf'] = 'YLR006C'

In [22]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [23]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Cells counted, # of Cells with Foci, % Cells with foci, ORF, Gene Name, Alias, Description, orf]
Index: []


In [24]:
original_data['data'] = pd.to_numeric(original_data['% Cells with foci'], errors='coerce')

In [25]:
original_data.set_index('orf', inplace=True)

In [26]:
original_data = original_data[['data']].copy()

In [27]:
original_data = original_data.groupby(original_data.index).mean()

In [28]:
original_data.shape

(4741, 1)

# Prepare the final dataset

In [29]:
data = original_data.copy()

In [30]:
dataset_ids = [591]
datasets = datasets.reindex(index=dataset_ids)

In [31]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [32]:
data.head()

dataset_id,591
data_type,value
orf,
YAL002W,1.142857
YAL004W,8.695652
YAL005C,3.529412
YAL007C,13.636364
YAL008W,6.250000


## Subset to the genes currently in SGD

In [33]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 17


In [34]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,591
,data_type,value
gene_id,orf,
2,YAL002W,1.142857
1863,YAL004W,8.695652
4,YAL005C,3.529412
5,YAL007C,13.636364
6,YAL008W,6.250000


# Normalize

In [35]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [36]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [37]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id             591          
data_type            value    valuez
gene_id orf                         
2       YAL002W   1.142857 -0.170328
1863    YAL004W   8.695652  1.080458
4       YAL005C   3.529412  0.224899
5       YAL007C  13.636364  1.898668
6       YAL008W   6.250000  0.675444

# Print out

In [38]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [39]:
from IO.save_data_to_db3 import *

In [40]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 18085829...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.24s/it]

Updating the data_modified_on field...
